# Construcción del dataset

In [1]:
import pandas as pd
from mlforecast import MLForecast
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np

from window_ops.rolling import rolling_mean
import optuna
from sklearn.model_selection import TimeSeriesSplit
from utilsforecast.feature_engineering import fourier

/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

df = pd.read_csv('../data/sell-in.txt', sep='\t', encoding='utf-8')
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [3]:
df_productos_predecir = pd.read_csv('../data/product_id_apredecir201912.txt', sep='\t', encoding='utf-8')
df_productos_predecir.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [4]:
df['periodo'].sort_values().unique()

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912])

In [5]:
df_pivot = df.pivot_table(
    index=['product_id', 'customer_id'],
    columns='periodo',
    values='tn',
    aggfunc='sum',
    fill_value=None
)
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None
df_pivot.head()

,product_id,customer_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,20001,10001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,20001,10002,35.72806,6.79415,29.94128,22.81133,31.22847,47.57025,21.84874,17.08052,...,31.97079,55.41679,30.87299,144.07021,37.14616,NaN,72.08551,17.40806,45.61495,113.33165
2,20001,10003,143.49426,20.48319,137.87537,68.89292,135.12190,171.01785,64.66196,83.63410,...,170.89924,230.00152,1.84835,NaN,138.23391,162.07198,233.20532,76.00625,86.14415,102.27517
3,20001,10004,184.72927,104.03894,295.43924,247.65632,188.37819,195.02683,379.44270,237.16848,...,102.64484,91.67799,389.02653,66.71971,228.62366,96.11402,288.34205,324.96172,195.67828,34.64810
4,20001,10005,19.08407,5.17117,5.17117,0.86186,37.95546,19.08407,43.35049,67.53856,...,6.90049,22.18016,15.89578,NaN,8.25595,NaN,12.80400,17.13921,12.22149,19.60368


In [6]:
# Remove from df_pivot the products that are not in df_productos_predecir
df_pivot = df_pivot[df_pivot['product_id'].isin(df_productos_predecir['product_id'])]

In [7]:
# df_mlforecast = df_pivot[df_pivot['customer_id'] == 10001].copy()
df_mlforecast = df_pivot.copy()

In [8]:
df_mlforecast.head()

,product_id,customer_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,20001,10001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,20001,10002,35.72806,6.79415,29.94128,22.81133,31.22847,47.57025,21.84874,17.08052,...,31.97079,55.41679,30.87299,144.07021,37.14616,NaN,72.08551,17.40806,45.61495,113.33165
2,20001,10003,143.49426,20.48319,137.87537,68.89292,135.12190,171.01785,64.66196,83.63410,...,170.89924,230.00152,1.84835,NaN,138.23391,162.07198,233.20532,76.00625,86.14415,102.27517
3,20001,10004,184.72927,104.03894,295.43924,247.65632,188.37819,195.02683,379.44270,237.16848,...,102.64484,91.67799,389.02653,66.71971,228.62366,96.11402,288.34205,324.96172,195.67828,34.64810
4,20001,10005,19.08407,5.17117,5.17117,0.86186,37.95546,19.08407,43.35049,67.53856,...,6.90049,22.18016,15.89578,NaN,8.25595,NaN,12.80400,17.13921,12.22149,19.60368


In [9]:
df_mlforecast.shape

(262805, 38)

In [10]:
# --- PASO 1: TRANSFORMACIÓN DE DATOS A FORMATO LARGO ---
# Este es el formato conveniente que usaremos en ambos casos.
print("\n--- 1. Transformando datos a formato largo ---")
df_long = df_mlforecast.melt(
    id_vars=['product_id', 'customer_id'],
    var_name='periodo',
    value_name='y' # MLForecast usa 'y' como nombre de la variable objetivo
)

df_long.head()


--- 1. Transformando datos a formato largo ---


,product_id,customer_id,periodo,y
0,20001,10001,201701,99.43861
1,20001,10002,201701,35.72806
2,20001,10003,201701,143.49426
3,20001,10004,201701,184.72927
4,20001,10005,201701,19.08407


In [11]:
FECHA_CORTE = '2019-10-01'
horizonte_prediccion = 2
# Lista de product_id únicos a procesar
product_ids = df_mlforecast['product_id'].unique()

# DataFrame para acumular resultados finales
df_pred_final = pd.DataFrame()

for pid in product_ids:
    # 1. Filtrar datos para el product_id actual
    df_pivot_prod = df_pivot[df_pivot['product_id'] == pid]
    if df_pivot_prod.empty:
        continue

    # 2. Transformar a formato largo
    df_long_prod = df_pivot_prod.melt(
        id_vars=['product_id', 'customer_id'],
        var_name='periodo',
        value_name='y'
    )
    df_long_prod = df_long_prod.fillna(0)
    df_long_prod['unique_id'] = df_long_prod['product_id'].astype(str) + "_" + df_long_prod['customer_id'].astype(str)
    df_long_prod['ds'] = pd.to_datetime(df_long_prod['periodo'], format='%Y%m')
    df_final_prod = df_long_prod[['unique_id', 'ds', 'y']].sort_values(by=['unique_id', 'ds']).reset_index(drop=True)

    # 3. Dividir en entrenamiento y validación
    df_final_prod = df_final_prod.loc[:, ~df_final_prod.columns.duplicated()]
    df_entrenamiento_prod = df_final_prod[df_final_prod['ds'] <= FECHA_CORTE]
    # Entrenar modelo
    fcst_prod = MLForecast(
        models=LGBMRegressor(random_state=42, n_estimators=100),
        freq='MS',
        lags=[1, 2, 3, 6, 12],
        date_features=['month', 'year'],
    )
    fcst_prod.fit(df_entrenamiento_prod, static_features=[])

    # 4. Predecir 2 pasos adelante (201911 y 201912)
    pred_prod = fcst_prod.predict(h=horizonte_prediccion)
    pred_prod['product_id'] = pid

    # 5. Filtrar solo 201912 y extraer customer_id
    pred_prod_201912 = pred_prod[pred_prod['ds'] == '2019-12-01'].copy()
    pred_prod_201912['customer_id'] = pred_prod_201912['unique_id'].str.split('_').str[1].astype(int)
    pred_prod_201912.rename(columns={'LGBMRegressor': 'tn'}, inplace=True)

    # 6. Acumular resultados
    df_pred_final = pd.concat([df_pred_final, pred_prod_201912[['product_id', 'customer_id', 'tn']]], ignore_index=True)

# Sumar tn por producto
df_pred_sum = df_pred_final.groupby('product_id', as_index=False)['tn'].sum()
print(df_pred_sum)


/tmp/ipykernel_1054430/558653976.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000769 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 9768, number of used features: 7
[LightGBM] [Info] Start training from score 3.340021


/tmp/ipykernel_1054430/558653976.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001831 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 9790, number of used features: 7
[LightGBM] [Info] Start training from score 2.517133


/tmp/ipykernel_1054430/558653976.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/tmp/ipykernel_1054430/558653976.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)
/tmp/ipykernel_1054430/558653976.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/tmp/ipykernel_1054430/558653976.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)
/tmp/ipykernel_1054430/558653976.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [12]:
df_pred_sum.head()

,product_id,tn
0,20001,1771.256376
1,20002,1433.880117
2,20003,1015.419283
3,20004,796.958411
4,20005,730.953320


In [13]:
# 1. Extraer los valores reales de df_long para 201912 y sumarlos por producto
df_validacion = (
    df_long[df_long['periodo'] == 201912]  # sin comillas, como int
    .groupby('product_id', as_index=False)['y']
    .sum()
    .rename(columns={'y': 'tn_real'})
)

# 2. Unir con las predicciones
df_eval = df_validacion.merge(df_pred_sum, on='product_id', how='inner')

y_real = df_eval['tn_real']
y_pred = df_eval['tn']

# 3. Calcular las métricas
mae = mean_absolute_error(y_real, y_pred)
mse = mean_squared_error(y_real, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_real, y_pred)
r2 = r2_score(y_real, y_pred)

# 4. Imprimir resultados
print("\n" + "="*40)
print(" MÉTRICAS DE RENDIMIENTO DEL MODELO")
print("="*40)
print(f"Error Absoluto Medio (MAE):       {mae:.2f} unidades")
print(f"Raíz del Error Cuadrático (RMSE): {rmse:.2f} unidades")
print(f"Error Porcentual Absoluto (MAPE): {mape:.2%}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")
print("="*40)

print("\nInterpretación:")
print(f"- En promedio, el modelo se equivoca en {mae:.2f} toneladas (o la unidad que estés usando).")
print(f"- El error porcentual promedio es de {mape:.2%}.")
print(f"- Un R² de {r2:.2f} indica qué proporción de la varianza de los datos es explicada por el modelo (más cercano a 1 es mejor).")



 MÉTRICAS DE RENDIMIENTO DEL MODELO
Error Absoluto Medio (MAE):       11.63 unidades
Raíz del Error Cuadrático (RMSE): 31.93 unidades
Error Porcentual Absoluto (MAPE): 250.65%
Coeficiente de Determinación (R²): 0.89

Interpretación:
- En promedio, el modelo se equivoca en 11.63 toneladas (o la unidad que estés usando).
- El error porcentual promedio es de 250.65%.
- Un R² de 0.89 indica qué proporción de la varianza de los datos es explicada por el modelo (más cercano a 1 es mejor).


In [14]:
FECHA_CORTE_FINAL = '2019-12-01'
horizonte_prediccion_final = 2
df_pred_final_202002 = pd.DataFrame()

product_ids = df_mlforecast['product_id'].unique()

for pid in product_ids:
    df_pivot_prod = df_pivot[df_pivot['product_id'] == pid]
    if df_pivot_prod.empty:
        continue

    df_long_prod = df_pivot_prod.melt(
        id_vars=['product_id', 'customer_id'],
        var_name='periodo',
        value_name='y'
    )
    df_long_prod = df_long_prod.fillna(0)
    df_long_prod['unique_id'] = df_long_prod['product_id'].astype(str) + "_" + df_long_prod['customer_id'].astype(str)
    df_long_prod['ds'] = pd.to_datetime(df_long_prod['periodo'], format='%Y%m')
    df_final_prod = df_long_prod[['unique_id', 'ds', 'y']].sort_values(by=['unique_id', 'ds']).reset_index(drop=True)

    df_final_prod = df_final_prod.loc[:, ~df_final_prod.columns.duplicated()]
    df_entrenamiento_prod = df_final_prod[df_final_prod['ds'] <= FECHA_CORTE_FINAL]

    fcst_prod = MLForecast(
        models=LGBMRegressor(random_state=42, n_estimators=100),
        freq='MS',
        lags=[1, 2, 3, 6, 12],
        date_features=['month', 'year'],
    )
    fcst_prod.fit(df_entrenamiento_prod, static_features=[])

    pred_prod = fcst_prod.predict(h=horizonte_prediccion_final)
    pred_prod['product_id'] = pid

    # Filtrar solo 2020-02-01 y extraer customer_id
    pred_prod_202002 = pred_prod[pred_prod['ds'] == '2020-02-01'].copy()
    pred_prod_202002['customer_id'] = pred_prod_202002['unique_id'].str.split('_').str[1].astype(int)
    pred_prod_202002.rename(columns={'LGBMRegressor': 'tn'}, inplace=True)

    df_pred_final_202002 = pd.concat([df_pred_final_202002, pred_prod_202002[['product_id', 'customer_id', 'tn']]], ignore_index=True)

# Sumar tn por producto para 202002
df_pred_sum_202002 = df_pred_final_202002.groupby('product_id', as_index=False)['tn'].sum()
print(df_pred_sum_202002)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001200 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 10656, number of used features: 7
[LightGBM] [Info] Start training from score 3.334027


/tmp/ipykernel_1054430/3187491485.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)
/tmp/ipykernel_1054430/3187491485.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001791 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 10680, number of used features: 7
[LightGBM] [Info] Start training from score 2.542473


/tmp/ipykernel_1054430/3187491485.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001173 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 9648, number of used features: 7
[LightGBM] [Info] Start training from score 2.064608


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000243 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 68
[LightGBM] [Info] Number of data points in the train set: 1440, number of used features: 6
[LightGBM] [Info] Start training from score 0.000622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

/tmp/ipykernel_1054430/3187491485.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/tmp/ipykernel_1054430/3187491485.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)
/tmp/ipykernel_1054430/3187491485.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long_prod = df_long_prod.fillna(0)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [15]:
df_pred_sum_202002.head()

,product_id,tn
0,20001,1373.414967
1,20002,958.390012
2,20003,706.461069
3,20004,505.657139
4,20005,550.054796


In [18]:
df_pred_sum_202002.shape

(780, 2)

In [19]:
df_pred_sum_202002.to_csv('prediccion_tn_por_producto_3.csv', index=False)